# Part 0. Data Prepraration

In [1]:
from datasets import load_dataset
dataset = load_dataset("rotten_tomatoes")
train_dataset = dataset ['train']
validation_dataset = dataset ['validation']
test_dataset = dataset ['test']

c:\Users\JJWX\anaconda3\envs\NLP\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Part 3. Enhancement


In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
import gensim.downloader as api
import numpy as np
import nltk
import random

In [3]:
model = api.load("glove-wiki-gigaword-100")
vocab_size = len(model.index_to_key) + 1
embedding_dim = model.vector_size
word_index = {word: index+1 for index, word in enumerate(model.index_to_key)} # index 0 is reserved for padding
embedding_matrix = np.zeros((vocab_size, embedding_dim))

vocab_size = len(model.index_to_key) + 1
embedding_dim = model.vector_size
word_index = {word: index+1 for index, word in enumerate(model.index_to_key)} # index 0 is reserved for padding
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, idx in word_index.items():
    if word in model:
        embedding_matrix[idx] = model[word]

def tokenize(text, word_index):
    ls = nltk.word_tokenize(text)
    return [word_index[word] for word in ls if word in word_index]

X_train = [tokenize(text, word_index) for text in train_dataset['text']]
X_val = [tokenize(text, word_index) for text in validation_dataset['text']]
X_test = [tokenize(text, word_index) for text in test_dataset['text']]
max_length = max(len(seq) for seq in X_train)

X_train = pad_sequences(X_train, maxlen=max_length, padding='pre', truncating='pre')
X_val = pad_sequences(X_val, maxlen=max_length, padding='pre', truncating='pre')
X_test = pad_sequences(X_test, maxlen=max_length, padding='pre', truncating='pre')

y_train = np.array(train_dataset['label'])
y_val = np.array(validation_dataset['label'])
y_test = np.array(test_dataset['label'])

#### 1. Instead of keeping the word embeddings fixed, now update the word embeddings (the same way as model parameters) during the training process.


In [4]:
tf.random.set_seed(0)
np.random.seed(0)
random.seed(0)
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=3,
    restore_best_weights=True
)
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="model_3_1.keras", 
    monitor='val_accuracy',            
    save_best_only=True,           
    mode='max',                 
    save_weights_only=False,       
    verbose=1
)
model = Sequential([
    Embedding(input_dim=vocab_size,
              output_dim=embedding_dim,
              weights=[embedding_matrix],
              trainable=True),
    SimpleRNN(16, return_sequences=False),
    Dense(1, activation='sigmoid')
])
optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.01)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=64,
    callbacks=[checkpoint_callback, early_stopping]
)

Epoch 1/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step - accuracy: 0.4895 - loss: 0.7142
Epoch 1: val_accuracy improved from -inf to 0.53189, saving model to model_3_1.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 24s 165ms/step - accuracy: 0.4896 - loss: 0.7141 - val_accuracy: 0.5319 - val_loss: 0.6920
Epoch 2/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step - accuracy: 0.5222 - loss: 0.6915
Epoch 2: val_accuracy improved from 0.53189 to 0.55629, saving model to model_3_1.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 21s 159ms/step - accuracy: 0.5222 - loss: 0.6914 - val_accuracy: 0.5563 - val_loss: 0.6864
Epoch 3/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step - accuracy: 0.5522 - loss: 0.6864
Epoch 3: val_accuracy did not improve from 0.55629
134/134 ━━━━━━━━━━━━━━━━━━━━ 20s 149ms/step - accuracy: 0.5522 - loss: 0.6864 - val_accuracy: 0.5553 - val_loss: 0.6833
Epoch 4/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step - accuracy: 0.5603 - loss: 0.6831
Epoch 4: val_accuracy improved from 0.55629 to 0.56004, savi

In [5]:
best_model = tf.keras.models.load_model("model_3_1.keras")
accuracy = best_model.evaluate(X_test, y_test)
print(f"Test accuracy: {accuracy[1] * 100:.2f}%")

34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6270 - loss: 0.6541
Test accuracy: 70.54%


#### 2. As discussed in Question 1(c), apply your solution in mitigating the influence of OOV words and train your model again.

In [6]:
model = api.load("glove-wiki-gigaword-100")
vocab_size = len(model.index_to_key) + 2 # 0 is reserved for padding, 1 is reserved for OOV
embedding_dim = model.vector_size
word_index = {word: index+2 for index, word in enumerate(model.index_to_key)} # index 0 is reserved for padding
embedding_matrix = np.zeros((vocab_size, embedding_dim))

In [7]:
for word, idx in word_index.items():
    if word in model:
        embedding_matrix[idx] = model[word]

In [8]:
def tokenize(text, word_index):
    ls = nltk.word_tokenize(text)
    return [word_index[word] if word in word_index else 1 for word in ls]

X_train = [tokenize(text, word_index) for text in train_dataset['text']]
X_val = [tokenize(text, word_index) for text in validation_dataset['text']]
X_test = [tokenize(text, word_index) for text in test_dataset['text']]
max_length = max(len(seq) for seq in X_train)

In [9]:
X_train = pad_sequences(X_train, maxlen=max_length)
X_val = pad_sequences(X_val, maxlen=max_length)
X_test = pad_sequences(X_test, maxlen=max_length)

In [10]:
y_train = np.array(train_dataset['label'])
y_val = np.array(validation_dataset['label'])
y_test = np.array(test_dataset['label'])

In [11]:
def train_model(optimizer, epochs, batch_size, lr):
    tf.random.set_seed(0)
    np.random.seed(0)
    random.seed(0)
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=3,
        restore_best_weights=True
    )
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath="model_oov.keras", 
        monitor='val_accuracy',            
        save_best_only=True,           
        mode='max',                 
        save_weights_only=False,       
        verbose=1
    )
    model = Sequential([
        Embedding(input_dim=vocab_size,
                  output_dim=embedding_dim,
                  weights=[embedding_matrix],
                  trainable=True), 
        SimpleRNN(16, return_sequences=False),
        Dense(1, activation='sigmoid')
    ])
    if optimizer == 'adam': optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    elif optimizer == 'sgd': optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
    elif optimizer == 'rmsprop': optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr)
    else: optimizer = tf.keras.optimizers.Adagrad(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[checkpoint_callback, early_stopping]
    )
    return model, history

In [12]:
model, history = train_model("adagrad", 100, 64, 0.01)

Epoch 1/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step - accuracy: 0.4908 - loss: 0.7134
Epoch 1: val_accuracy improved from -inf to 0.53846, saving model to model_oov.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 23s 162ms/step - accuracy: 0.4909 - loss: 0.7133 - val_accuracy: 0.5385 - val_loss: 0.6900
Epoch 2/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step - accuracy: 0.5255 - loss: 0.6912
Epoch 2: val_accuracy improved from 0.53846 to 0.56848, saving model to model_oov.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 22s 164ms/step - accuracy: 0.5256 - loss: 0.6912 - val_accuracy: 0.5685 - val_loss: 0.6848
Epoch 3/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step - accuracy: 0.5528 - loss: 0.6862
Epoch 3: val_accuracy improved from 0.56848 to 0.57411, saving model to model_oov.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 21s 159ms/step - accuracy: 0.5528 - loss: 0.6862 - val_accuracy: 0.5741 - val_loss: 0.6819
Epoch 4/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step - accuracy: 0.5625 - loss: 0.6829
Epoch 4: val_accuracy d

In [13]:
best_model = tf.keras.models.load_model("model_oov.keras")
accuracy = best_model.evaluate(X_test, y_test)
print(f"Test accuracy: {accuracy[1] * 100:.2f}%")

34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5742 - loss: 0.6902
Test accuracy: 56.10%


#### 3. Keeping the above two adjustments, replace your simple RNN model in Part 2 with a biLSTM model and a biGRU model, incorporating recurrent computations in both directions and stacking multiple layers if possible

BiLSTM

In [14]:
from tensorflow.keras.layers import Bidirectional, LSTM
def train_model(optimizer, epochs, batch_size, lr):
    tf.random.set_seed(0)
    np.random.seed(0)
    random.seed(0)
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=3,
        restore_best_weights=True
    )
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath="model_bilstm.keras", 
        monitor='val_accuracy',            
        save_best_only=True,           
        mode='max',                 
        save_weights_only=False,       
        verbose=1
    )
    model = Sequential([
        Embedding(input_dim=vocab_size,
                  output_dim=embedding_dim,
                  weights=[embedding_matrix],
                  trainable=True),
        Bidirectional(LSTM(16, return_sequences=True)),
        Bidirectional(LSTM(16, return_sequences=True)),
        Bidirectional(LSTM(16, return_sequences=False)),
        Dense(1, activation='sigmoid')
    ])
    if optimizer == 'adam': optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    elif optimizer == 'sgd': optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
    elif optimizer == 'rmsprop': optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr)
    else: optimizer = tf.keras.optimizers.Adagrad(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[checkpoint_callback, early_stopping]
    )
    return model, history

In [15]:
model, history = train_model("adagrad", 100, 64, 0.01)

Epoch 1/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step - accuracy: 0.4913 - loss: 0.6940
Epoch 1: val_accuracy improved from -inf to 0.53846, saving model to model_bilstm.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 35s 200ms/step - accuracy: 0.4913 - loss: 0.6940 - val_accuracy: 0.5385 - val_loss: 0.6921
Epoch 2/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step - accuracy: 0.5293 - loss: 0.6917
Epoch 2: val_accuracy improved from 0.53846 to 0.55722, saving model to model_bilstm.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 27s 203ms/step - accuracy: 0.5293 - loss: 0.6917 - val_accuracy: 0.5572 - val_loss: 0.6909
Epoch 3/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step - accuracy: 0.5537 - loss: 0.6900
Epoch 3: val_accuracy improved from 0.55722 to 0.56567, saving model to model_bilstm.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 27s 203ms/step - accuracy: 0.5537 - loss: 0.6900 - val_accuracy: 0.5657 - val_loss: 0.6891
Epoch 4/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step - accuracy: 0.5722 - loss: 0.6874
Epoch 4: val_a

In [16]:
best_model = tf.keras.models.load_model("model_bilstm.keras")
accuracy = best_model.evaluate(X_test, y_test)
print("Test accuracy:", accuracy[1])

34/34 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.5844 - loss: 0.7106
Test accuracy: 0.7204502820968628


BiGRU

In [17]:
from tensorflow.keras.layers import Bidirectional, GRU
def train_model(optimizer, epochs, batch_size, lr):
    tf.random.set_seed(0)
    np.random.seed(0)
    random.seed(0)
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=3,
        restore_best_weights=True
    )
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath="model_bigru.keras", 
        monitor='val_accuracy',            
        save_best_only=True,           
        mode='max',                 
        save_weights_only=False,       
        verbose=1
    )
    model = Sequential([
        Embedding(input_dim=vocab_size,
                  output_dim=embedding_dim,
                  weights=[embedding_matrix],
                  trainable=True), 
        Bidirectional(GRU(16, return_sequences=False)),
        Dense(1, activation='sigmoid')
    ])
    if optimizer == 'adam': optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    elif optimizer == 'sgd': optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
    elif optimizer == 'rmsprop': optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr)
    else: optimizer = tf.keras.optimizers.Adagrad(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[checkpoint_callback, early_stopping]
    )
    return model, history

In [18]:
model, history = train_model("adagrad", 100, 64, 0.01)

Epoch 1/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step - accuracy: 0.5123 - loss: 0.6944
Epoch 1: val_accuracy improved from -inf to 0.51876, saving model to model_bigru.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 32s 198ms/step - accuracy: 0.5122 - loss: 0.6944 - val_accuracy: 0.5188 - val_loss: 0.6910
Epoch 2/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step - accuracy: 0.5373 - loss: 0.6895
Epoch 2: val_accuracy improved from 0.51876 to 0.54034, saving model to model_bigru.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 24s 182ms/step - accuracy: 0.5373 - loss: 0.6895 - val_accuracy: 0.5403 - val_loss: 0.6876
Epoch 3/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step - accuracy: 0.5605 - loss: 0.6857
Epoch 3: val_accuracy improved from 0.54034 to 0.56379, saving model to model_bigru.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 24s 176ms/step - accuracy: 0.5605 - loss: 0.6857 - val_accuracy: 0.5638 - val_loss: 0.6842
Epoch 4/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step - accuracy: 0.5783 - loss: 0.6818
Epoch 4: val_accu

In [19]:
best_model = tf.keras.models.load_model("model_bigru.keras")
accuracy = best_model.evaluate(X_test, y_test)
print("Test accuracy:", accuracy[1])

34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6402 - loss: 0.6373
Test accuracy: 0.7307692170143127


#### 4. Keeping the above two adjustments, replace your simple RNN model in Part 2 with a Convolutional Neural Network (CNN) to produce sentence representations and perform sentiment classification

In [20]:
from tensorflow.keras.layers import Convolution1D, Flatten

def train_model(optimizer, epochs, batch_size, lr):
    tf.random.set_seed(0)
    np.random.seed(0)
    random.seed(0)
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=3,
        restore_best_weights=True
    )
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath="model_cnn.keras", 
        monitor='val_accuracy',            
        save_best_only=True,           
        mode='max',                 
        save_weights_only=False,       
        verbose=1
    )
    model = Sequential([
        Embedding(input_dim=vocab_size,
                  output_dim=embedding_dim,
                  weights=[embedding_matrix],
                  trainable=True), 
        Convolution1D(16, kernel_size=3, activation='relu'),
        Flatten(),
        Dense(1, activation='sigmoid')
    ])
    if optimizer == 'adam': optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    elif optimizer == 'sgd': optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
    elif optimizer == 'rmsprop': optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr)
    else: optimizer = tf.keras.optimizers.Adagrad(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[checkpoint_callback, early_stopping]
    )
    return model, history

In [21]:
model, history = train_model("adagrad", 100, 64, 0.01)

Epoch 1/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step - accuracy: 0.5202 - loss: 0.6951
Epoch 1: val_accuracy improved from -inf to 0.57129, saving model to model_cnn.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 26s 175ms/step - accuracy: 0.5202 - loss: 0.6951 - val_accuracy: 0.5713 - val_loss: 0.6835
Epoch 2/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step - accuracy: 0.5760 - loss: 0.6784
Epoch 2: val_accuracy improved from 0.57129 to 0.60882, saving model to model_cnn.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 36s 269ms/step - accuracy: 0.5761 - loss: 0.6783 - val_accuracy: 0.6088 - val_loss: 0.6708
Epoch 3/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step - accuracy: 0.6335 - loss: 0.6533
Epoch 3: val_accuracy improved from 0.60882 to 0.62946, saving model to model_cnn.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 21s 160ms/step - accuracy: 0.6335 - loss: 0.6532 - val_accuracy: 0.6295 - val_loss: 0.6594
Epoch 4/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step - accuracy: 0.6662 - loss: 0.6247
Epoch 4: val_accuracy i

In [22]:
best_model = tf.keras.models.load_model("model_cnn.keras")
accuracy = best_model.evaluate(X_test, y_test)
print("Test accuracy:", accuracy[1])

34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6685 - loss: 0.6602
Test accuracy: 0.7326453924179077


#### 5. Further improve your model. You are free to use any strategy other than the above mentioned solutions. Changing hyper-parameters or stacking more layers is not counted towards a meaningful improvement.

In [ ]:
from tensorflow.keras.layers import MultiHeadAttention, Input, Bidirectional, Dropout, Dense, GlobalMaxPooling1D, GRU
from tensorflow.keras.models import Model

def train_model(optimizer, epochs, batch_size, lr):
    tf.random.set_seed(0)
    np.random.seed(0)
    random.seed(0)
    
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath="model_combined.keras", 
        monitor='val_accuracy',            
        save_best_only=True,           
        mode='max',                 
        save_weights_only=False,       
        verbose=1
    )

    input_layer = Input(shape=(max_length,))
    embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim, weights=[embedding_matrix], trainable=True)(input_layer)
    
    gru_output = GRU(32, return_sequences=True)(embedding_layer)
    gru_output = Dropout(0.5)(gru_output)
    attention_output = MultiHeadAttention(num_heads=1, key_dim=16)(gru_output, gru_output)
    gru_output = Bidirectional(GRU(16, return_sequences=True))(attention_output)
    max_output = GlobalMaxPooling1D()(gru_output)
    gru_output = gru_output[:, -1, :]
    concat_output = tf.keras.layers.Concatenate(axis=1)([max_output, gru_output])
    output_layer = Dense(1, activation='sigmoid')(concat_output) 

    # Create the model
    model = Model(inputs=input_layer, outputs=output_layer)

    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',    
        factor=0.75,             
        patience=3,            
        min_lr=1e-6             
    )
    # Set optimizer
    if optimizer == 'adam': optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    elif optimizer == 'sgd': optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
    elif optimizer == 'rmsprop': optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr)
    else: optimizer = tf.keras.optimizers.Adagrad(learning_rate=lr)

    # Compile the model
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    # Train the model
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[checkpoint_callback, reduce_lr]
    )

    return model, history

In [ ]:
model, history = train_model("adagrad", 200, 64, 0.01)

Epoch 1/200
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.4811 - loss: 0.6937
Epoch 1: val_accuracy improved from -inf to 0.50000, saving model to model_combined.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 13s 69ms/step - accuracy: 0.4811 - loss: 0.6937 - val_accuracy: 0.5000 - val_loss: 0.6930 - learning_rate: 0.0100
Epoch 2/200
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.5062 - loss: 0.6929
Epoch 2: val_accuracy did not improve from 0.50000
134/134 ━━━━━━━━━━━━━━━━━━━━ 8s 62ms/step - accuracy: 0.5062 - loss: 0.6929 - val_accuracy: 0.4981 - val_loss: 0.6928 - learning_rate: 0.0100
Epoch 3/200
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.5187 - loss: 0.6926
Epoch 3: val_accuracy improved from 0.50000 to 0.51407, saving model to model_combined.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 9s 66ms/step - accuracy: 0.5186 - loss: 0.6926 - val_accuracy: 0.5141 - val_loss: 0.6925 - learning_rate: 0.0100
Epoch 4/200
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.5292 - 

In [ ]:
model = tf.keras.models.load_model("model_combined.keras")
accuracy = model.evaluate(X_test, y_test)
print("Test accuracy:", accuracy[1])

34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7519 - loss: 0.4918
Test accuracy: 0.7729831337928772
